In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"david123web","key":"ff634a92806dd060d9cda9630b0a75ed"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json


     |████████████████████████████████| 61kB 4.6MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=e9ec545f2e97df34bb9dc7750ca0a6819c37434616ddd2dcb1f5b17ed4cecf14
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 98% 280M/285M [00:02<00:00, 134MB/s]
100% 285M/285M [00:02<00:00, 108MB/s]


In [ ]:
!unzip /content/challenges-in-representation-learning-facial-expression-recognition-challenge.zip
!rm -rf /content/train.csv /content/test.csv /content/icml_face_data.csv /content/example_submission.csv /content/challenges-in-representation-learning-facial-expression-recognition-challenge.zip
!tar -xzf /content/fer2013.tar.gz
!mkdir dataset
!mkdir dataset/fer2013
!mv /content/fer2013/fer2013.csv /content/dataset/fer2013
!rm -rf /content/fer2013 /content/fer2013.tar.gz

Archive:  /content/challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [ ]:

!pip uninstall --yes keras_vggface
!pip uninstall --yes keras
!pip install keras
!pip install keras_vggface
!pip install pandas
!pip install scikit_image
!pip install h5py
!pip install keras_applications

Uninstalling keras-vggface-0.6:
  Successfully uninstalled keras-vggface-0.6
Uninstalling Keras-2.4.3:
  Successfully uninstalled Keras-2.4.3
  Using cached https://files.pythonhosted.org/packages/44/e1/dc0757b20b56c980b5553c1b5c4c32d378c7055ab7bfa92006801ad359ab/Keras-2.4.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2f/7d/5f0319ebdc09ac1a2272364fa9583f5067b6f8aff93fbbf8835d81cbaad7/keras_vggface-0.6-py3-none-any.whl


In [ ]:
import pandas as pd
import torch
import numpy as np
import cv2
import numpy as np
import pandas as pd
from tensorflow.python.lib.io import file_io
from skimage.transform import resize
from keras import backend as K
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.models import Model
from keras.layers import Flatten, Dense 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, Callback
import h5py # For saving the model
%matplotlib inline

In [ ]:
resnet=ResNet50(include_top=False, pooling='none', input_shape=(64,64,3), weights='imagenet')

In [ ]:
x=Flatten()(resnet.output)

x= Dense(1024, activation='relu')(x)

preds=Dense(7, activation='softmax')(x)

model=Model(inputs=resnet.input, outputs=preds)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
class dataset_provider(object):
    def __init__(self, dataset_name, image_size):
        self.dataset_name = dataset_name
        self.image_size = image_size
        self.dataset_path = '/content/dataset/fer2013/fer2013.csv'

    def dataset_initializer(self):
        data = pd.read_csv(self.dataset_path)
        pixels = data['pixels'].tolist()
        width = 48
        height = 48
        faces = np.empty((len(data), 64, 64,3))
        i=0
        for pixel_sequence in pixels:
            face = [float(pixel) for pixel in pixel_sequence.split(' ')]
            face = np.asarray(face).reshape(width, height)
            face = resize(face, self.image_size, order =3, mode='constant')
            ret=np.empty((64, 64, 3))
            ret[:,:,0] =face
            ret[:,:,1] =face
            ret[:,:,2] =face
            faces[i, :,:,:]=ret
            i +=1

        faces = pre_process(faces)
        emotions = pd.get_dummies(data['emotion']).values
        return faces, emotions

def get_labels(dataset_name):
    if dataset_name == 'fer2013':
        return {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'sad', 5: 'surprise', 6: 'neutral'}
    else:
        raise Exception('Invalid args: Wrong dataset name.')


def get_class_to_arg(dataset_name):
    if dataset_name == 'fer2013':
        return {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'sad': 4, 'surprise': 5, 'neutral': 6}
    else:
        raise Exception('Invalid args: Wrong dataset name.')


def pre_process(x, _is=True):
    x = x.astype('float32')
    x = x / 255.0
    if _is:
        x = x - 0.5
        x = x * 2.0
    return x

In [ ]:
data = pd.read_csv('/content/dataset/fer2013/fer2013.csv')
data

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [ ]:
input_shape = (64, 64, 3)
dataset = dataset_provider('fer2013', image_size=input_shape[:2])
faces, emotions = dataset.dataset_initializer()
faces = pre_process(faces)
n_train = int(0.8 * len(faces))
n_test = len(faces) - n_train
ds_train_faces= faces[:n_train]
ds_train_emotions=emotions[:n_train]
ds_test_faces=faces[n_train:]
ds_test_emotions=emotions[n_train:]
val_data=(ds_test_faces,ds_test_emotions)

In [ ]:
ds_train_faces[0]
ds_train_faces[0].shape

(64, 64, 3)

In [ ]:
ds_train_faces[0]

array([[[-1.0042455, -1.0042455, -1.0042455],
        [-1.0033237, -1.0033237, -1.0033237],
        [-1.003165 , -1.003165 , -1.003165 ],
        ...,
        [-1.005283 , -1.005283 , -1.005283 ],
        [-1.0054166, -1.0054166, -1.0054166],
        [-1.0057396, -1.0057396, -1.0057396]],

       [[-1.0038522, -1.0038522, -1.0038522],
        [-1.0032196, -1.0032196, -1.0032196],
        [-1.0034151, -1.0034151, -1.0034151],
        ...,
        [-1.0045705, -1.0045705, -1.0045705],
        [-1.0047747, -1.0047747, -1.0047747],
        [-1.0053506, -1.0053506, -1.0053506]],

       [[-1.0044441, -1.0044441, -1.0044441],
        [-1.0042636, -1.0042636, -1.0042636],
        [-1.0045793, -1.0045793, -1.0045793],
        ...,
        [-1.0044444, -1.0044444, -1.0044444],
        [-1.0045643, -1.0045643, -1.0045643],
        [-1.0053148, -1.0053148, -1.0053148]],

       ...,

       [[-1.0031737, -1.0031737, -1.0031737],
        [-1.0026622, -1.0026622, -1.0026622],
        [-1.0032984, -

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

#callbacks
early_stop=EarlyStopping('val_loss', patience=50)
reduce_lr=ReduceLROnPlateau('val_loss', factor=0.1, patience=int(50/4), verbose=1)
callbacks=[early_stop, reduce_lr]
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        fill_mode ='reflect',
                        shear_range =10,
                        horizontal_flip=True)

# model parameters/compilation


for layer in resnet.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr = 1e-3, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0), loss='categorical_crossentropy',
              metrics=['accuracy'])

# loading dataset
model.fit_generator(data_generator.flow(ds_train_faces, ds_train_emotions,
                                            32),
                        steps_per_epoch=len(ds_train_faces) / 32,
                        epochs=5, verbose=1,callbacks=callbacks, validation_data=val_data)



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
897/897 [==============================] - 80s 50ms/step - loss: 1.8943 - accuracy: 0.2137 - val_loss: 1.8139 - val_accuracy: 0.2471
Epoch 2/5
897/897 [==============================] - 44s 49ms/step - loss: 1.8124 - accuracy: 0.2518 - val_loss: 1.8183 - val_accuracy: 0.2471
Epoch 3/5
897/897 [==============================] - 44s 49ms/step - loss: 1.8160 - accuracy: 0.2498 - val_loss: 1.8159 - val_accuracy: 0.2471
Epoch 4/5
897/897 [==============================] - 44s 49ms/step - loss: 1.8173 - accuracy: 0.2498 - val_loss: 1.8165 - val_accuracy: 0.2471
Epoch 5/5
897/897 [==============================] - 44s 49ms/step - loss: 1.8147 - accuracy: 0.2493 - val_loss: 1.8159 - val_accuracy: 0.2471


In [ ]:
for layer in model.layers:
    layer.trainable = True

early_stop=EarlyStopping('val_loss', patience=10, mode='auto')
reduce_lr_plateau=ReduceLROnPlateau('val_loss', factor=0.5, patience=3,mode='auto', min_lr=1e-8)
def scheduler(epoch):
    updated_lr = K.get_value(model.optimizer.lr) * 0.5
    if (epoch % 3 == 0) and (epoch != 0):
        K.set_value(model.optimizer.lr, updated_lr)
    return K.get_value(model.optimizer.lr)


reduce_lr = LearningRateScheduler(scheduler)

callbacks=[early_stop, reduce_lr, reduce_lr_plateau]


model.compile( optimizer   = SGD(lr = 1e-4, momentum = 0.9, decay = 0.0, nesterov = True),
    loss        = 'categorical_crossentropy', 
    metrics     = ['accuracy'])

# loading dataset
model.fit_generator(data_generator.flow(ds_train_faces, ds_train_emotions,
                                            32),
                        steps_per_epoch=len(ds_train_faces) / 32,
                        epochs=100, verbose=1,callbacks=callbacks, validation_data=val_data)




Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
897/897 [==============================] - 70s 72ms/step - loss: 1.9153 - accuracy: 0.2502 - val_loss: 6.6468 - val_accuracy: 0.2471
Epoch 2/100
897/897 [==============================] - 65s 72ms/step - loss: 1.7178 - accuracy: 0.3053 - val_loss: 2.9269 - val_accuracy: 0.1158
Epoch 3/100
897/897 [==============================] - 64s 72ms/step - loss: 1.6947 - accuracy: 0.3127 - val_loss: 2.9416 - val_accuracy: 0.1158
Epoch 4/100
897/897 [==============================] - 66s 73ms/step - loss: 1.6854 - accuracy: 0.3203 - val_loss: 1.7954 - val_accuracy: 0.2973
Epoch 5/100
897/897 [==============================] - 66s 73ms/step - loss: 1.6651 - accuracy: 0.3290 - val_loss: 57.6873 - val_accuracy: 0.2471
Epoch 6/100
897/897 [==============================] - 65s 73ms/step - loss: 1.6453 - accuracy: 0.3383 - val_loss: 3.0555 - val_accuracy: 0.1158
Epoch 7/100
897/897 [==============================] - 65s 73ms/step - loss: 1.6244 - accuracy: 0.3507 - val_loss: 17.5199 - val_

In [ ]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")


In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
